# SEP Tutorial

In [ ]:
import numpy as np
import sep

Import Astropy to use as a FITS analyzer

In [ ]:
import astropy.io
from astropy.io import fits
import matplotlib.pyplot as plt 
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]

Open the FITS file and read it as a 2-d numpy array by indexing the image list

In [ ]:
image = fits.open("image.fits")
data = image[0].data

Display the image using the mean and standard deviation

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray',vmin=m-s,vmax=m+s, origin='lower')
plt.colorbar();

Determine a spatially varying background on the image file

In [ ]:
bkg = sep.Background(data)

Determine the 'global' mean and noise of the background created

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Convert the background to be a 2-d array -- the same size as the original image

In [ ]:
bkg_image = np.array(bkg)

Display the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Convert the background noise to be a 2-d array -- the same size as the original image and the background

In [ ]:
bkg_rms = bkg.rms()

Display the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Subtract the background from the data array in order for sources to be detected

In [ ]:
data_sub = data - bkg

Run object detection with the detection threshold to be 1.5 times the global background RMS

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Display the objects detected

In [ ]:
len(objects)

Plot the new background-subtracted image and a red ellipse for each object detected

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub,interpolation='nearest', cmap='gray', 
vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6*objects['a'][i],
               height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

Display the field types of the objects

In [ ]:
objects.dtype.names

Perform simple circular aperture photometry with 3 pixel radii at the object locations

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

Display the first 10 results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))